#Introduction

> This is my Notebook to analyze using LDA to topic model the reviews to identify the different topics of reviews across different genres and years

## Import Necessary Libraries

In [3]:
import pandas as pd
import re

----

## Step 1: Loading the Datasets

> First, we need to load the datasets and examine their structure.

In [ ]:
# Load the main dataset
df = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/preprocessed_reviews.csv')

<ipython-input-3-9626cd0b7050>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/preprocessed_reviews.csv')


In [ ]:
# Display the first few rows of the dataframe
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',..."
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'..."
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov..."
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '..."
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin..."


---

##Step 2: Loading Additional Datasets and Extracting IMDB IDs

> Next, we load the additional datasets and extract the IMDB IDs from the URLs.



In [ ]:
# Load the additional datasets
df2 = pd.read_csv('/content/drive/MyDrive/IMDB Project/non_review_analysis/data/without_reviews_by_budget.csv')
df3 = pd.read_csv('/content/drive/MyDrive/IMDB Project/Scraping/scraped_data/ScraPy_Code_1_data/all_movie_ids_final.csv')

<ipython-input-5-764384304640>:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('/content/drive/MyDrive/IMDB Project/non_review_analysis/data/without_reviews_by_budget.csv')


In [ ]:
# Extract the imdb_id from the details_url column in df3
df3['imdb_id'] = df3['details_url'].apply(lambda x: re.search('tt\d+', x).group(0))

# Display the first few rows of the dataframe
df3.head()

,Title,Gross,details_url,Genres,Year,imdb_id
0,Top Gun: Maverick,718732821,/title/tt1745960/,"\nAction, Drama",2023,tt1745960
1,Avatar: The Way of Water,659682302,/title/tt1630029/,"\nAction, Adventure, Fantasy",2023,tt1630029
2,Black Panther: Wakanda Forever,453721831,/title/tt9114286/,"\nAction, Adventure, Drama",2023,tt9114286
3,Doctor Strange in the Multiverse of Madness,411331607,/title/tt9419884/,"\nAction, Adventure, Fantasy",2023,tt9419884
4,Jurassic World Dominion,376851080,/title/tt8041270/,"\nAction, Adventure, Sci-Fi",2023,tt8041270


---

##Step 3: Merging Datasets
>Now, we merge the datasets based on the IMDB IDs.

In [ ]:
# Merge the datasets on the imdb_id column
df = pd.merge(df, df2[['imdb_id', 'release_date']], on='imdb_id', how='left')
df = pd.merge(df, df3[['imdb_id', 'Genres']], on='imdb_id', how='left')

In [ ]:
# Display the first few rows of the merged dataframe
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date_x,release_date_y,Genres
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,1995-08-05,"\nAction, Crime, Thriller"
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,1995-08-05,"\nAction, Crime, Thriller"
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,1995-08-05,"\nAction, Crime, Thriller"
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,1995-08-05,"\nAction, Crime, Thriller"
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,1995-08-05,"\nAction, Crime, Thriller"


In [ ]:
#There's a weird \n in the Genres column, let's remove it

def remove_newline(df, column_name):
    df[column_name] = df[column_name].str.replace('\n', '')
    return df

df['Genres'] = df['Genres'].astype(str).str.replace('\n', '')


In [ ]:
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date_x,release_date_y,Genres
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,1995-08-05,"Action, Crime, Thriller"


---

##Step 4: Saving the Merged Dataset
>After merging, we save the new dataset to a CSV file.

In [1]:
# Save the merged dataframe to a new CSV file
df.to_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/merged_data_pre_LDA.csv', index=False)

NameError: ignored

---

###Checkpoint for rerunning LDA


In [4]:
#Run this if reinstancing the environment and don't want to go through the above steps:
df_loaded = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/merged_data_pre_LDA.csv')

<ipython-input-4-24c3daef8f3d>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loaded = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/merged_data_pre_LDA.csv')


In [5]:
#Make a copy for editing
df = df_loaded.copy()

---

In [ ]:
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date_x,release_date_y,Genres
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,1995-08-05,"Action, Crime, Thriller"
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,1995-08-05,"Action, Crime, Thriller"


In [6]:
# There are 2 release dates, time to drop one of them and rename the other one
df.drop('release_date_y', axis=1, inplace=True)
df.rename(columns={'release_date_x': 'release_date'}, inplace=True)

In [ ]:
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date,Genres
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,"Action, Crime, Thriller"
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,"Action, Crime, Thriller"
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,"Action, Crime, Thriller"
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,"Action, Crime, Thriller"
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,"Action, Crime, Thriller"


---

##Step 5: Preparing for LDA - By Date
> Before we perform LDA, we need to prepare our data. Since you want to perform LDA by year and by genre, we need to group our data accordingly.

> We'll start with grouping by every 6 months

In [7]:
import numpy as np

# Convert 'release_date' to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

In [8]:
# Extract year and half-year from 'release_date'
df['year'] = df['release_date'].dt.year
df['half_year'] = np.where(df['release_date'].dt.month > 6, 2, 1)

# Create a new column 'year_half' to represent each 6-month period
df['year_half'] = df['year'].astype(str) + '-' + df['half_year'].astype(str)

# Display the first few rows of the dataframe
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date,Genres,year,half_year,year_half
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,"Action, Crime, Thriller",1995.0,2,1995.0-2
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,"Action, Crime, Thriller",1995.0,2,1995.0-2
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,"Action, Crime, Thriller",1995.0,2,1995.0-2
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,"Action, Crime, Thriller",1995.0,2,1995.0-2
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,"Action, Crime, Thriller",1995.0,2,1995.0-2


---

##Step 6: Performing LDA by Year
> Now, we perform LDA for each year.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [ ]:
# I like using this library to track the progress of each of my code executions
from tqdm import tqdm

In [ ]:
# Initialize a CountVectorizer to convert the text data into a bag-of-words
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)

In [ ]:
# Group the dataframe by 'year_half'
grouped = df.groupby('year_half')

# For each 6-month period, perform LDA
for name, group in tqdm(grouped):
    print(f'Performing LDA for 6-month period: {name}')

    # Fit and transform the processed reviews
    count_data = count_vectorizer.fit_transform(group['preprocessed_review'])

    # Initialize the LDA model
    lda = LDA(n_components=10, random_state=42)

    # Fit the model to the count data
    lda.fit(count_data)

    # Save the LDA model
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/lda_{name}.pkl', 'wb') as file:
        pickle.dump(lda, file)

Oops, seems like it didn't complete running from 2015 onwards.

In [ ]:
# Initialize a CountVectorizer to convert the text data into a bag-of-words
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)


# Group the dataframe by 'year_half'
grouped = df.groupby('year_half')

# For each 6-month period, perform LDA
for name, group in tqdm(grouped):
    print(f'Performing LDA for 6-month period: {name}')

    # Fit and transform the processed reviews
    count_data = count_vectorizer.fit_transform(group['preprocessed_review'])

    # Initialize the LDA model
    lda = LDA(n_components=10, random_state=42)

    # Fit the model to the count data
    lda.fit(count_data)

    # Save the LDA model
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/lda_{name}.pkl', 'wb') as file:
        pickle.dump(lda, file)


  0%|          | 0/19 [00:00<?, ?it/s]

Performing LDA for 6-month period: 2015.0-1


  5%|▌         | 1/19 [04:03<1:13:02, 243.49s/it]

Performing LDA for 6-month period: 2015.0-2


 11%|█         | 2/19 [08:43<1:15:08, 265.21s/it]

...

Performing LDA for 6-month period: 2023.0-1


 89%|████████▉ | 17/19 [1:47:50<11:42, 351.26s/it]

Performing LDA for 6-month period: 2023.0-2


100%|██████████| 19/19 [1:48:07<00:00, 341.46s/it]

Performing LDA for 6-month period: 2024.0-1


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import pickle
from tqdm import tqdm


# Filter the dataframe to only include rows from 2015 and later
df = df[df['year'] >= 2023]

# Initialize a CountVectorizer to convert the text data into a bag-of-words
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)

# Group the dataframe by 'year_half'
grouped = df.groupby('year_half')

# For each 6-month period, perform LDA
for name, group in tqdm(grouped):
    print(f'Performing LDA for 6-month period: {name}')

    # Fit and transform the processed reviews
    count_data = count_vectorizer.fit_transform(group['preprocessed_review'])

    # Save the feature names
    feature_names = count_vectorizer.get_feature_names_out()
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/feature_names_{name}.pkl', 'wb') as file:
        pickle.dump(feature_names, file)

    # Initialize the LDA model
    lda = LDA(n_components=7, random_state=42)

    # Fit the model to the count data
    lda.fit(count_data)

    # Save the LDA model
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/lda_{name}.pkl', 'wb') as file:
        pickle.dump(lda, file)


  0%|          | 0/3 [00:00<?, ?it/s]

Performing LDA for 6-month period: 2023.0-1


 33%|███▎      | 1/3 [03:26<06:53, 206.92s/it]

Performing LDA for 6-month period: 2023.0-2


100%|██████████| 3/3 [03:43<00:00, 74.64s/it]

Performing LDA for 6-month period: 2024.0-1


---

##Step 7: Analyzing Topic Distributions




In [ ]:
# Get a list of all pickle files
pickle_files = os.listdir('/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models')
print(pickle_files)

# For each pickle file
for file in pickle_files:
    if 'lda' in file:
        # Load the LDA model
        with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/{file}', 'rb') as f:
            lda = pickle.load(f)

        # Load the feature names
        with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/feature_names_{file[4:]}', 'rb') as f:
            feature_names = pickle.load(f)

        # Get the topic-word matrix
        topic_word_matrix = lda.components_

        # Plot the distribution for each topic
        for i in range(topic_word_matrix.shape[0]):
            # Create a dataframe for the current topic
            df_topic = pd.DataFrame({'Word': feature_names, 'Importance': topic_word_matrix[i, :]})

            # Sort the dataframe by importance
            df_topic = df_topic.sort_values('Importance', ascending=False)

            # Create a bar plot using Plotly
            fig = px.bar(df_topic.head(10), x='Word', y='Importance', title=f'Topic {i} Word Distribution for {file}')
            fig.show()


['feature_names_2022.0-1.pkl', 'feature_names_2023.0-1.pkl', 'lda_2023.0-1.pkl', 'feature_names_2023.0-2.pkl', 'lda_2023.0-2.pkl', 'feature_names_2024.0-1.pkl', 'lda_2024.0-1.pkl']


In [ ]:
import pandas as pd
import os
import pickle

# Get a list of all pickle files
pickle_files = os.listdir('/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models')

# For each pickle file
for file in pickle_files:
    if 'lda' in file:
        # Load the LDA model
        with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/{file}', 'rb') as f:
            lda = pickle.load(f)

        # Load the feature names
        with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/feature_names_{file[4:]}', 'rb') as f:
            feature_names = pickle.load(f)

        # Get the topic-word matrix
        topic_word_matrix = lda.components_

        # For each topic
        for i in range(topic_word_matrix.shape[0]):
            # Create a dataframe for the current topic
            df_topic = pd.DataFrame({'Word': feature_names, 'Importance': topic_word_matrix[i, :]})

            # Sort the dataframe by importance
            df_topic = df_topic.sort_values('Importance', ascending=False)

            # Print the dataframe
            print(f'Topic {i} Word Distribution for {file}')
            print(df_topic.head(5).to_string(index=False))


Topic 0 Word Distribution for lda_2023.0-1.pkl
     Word  Importance
    movie 7654.035112
     like 6740.318540
     film 6075.353710
character 3990.202746
   really 3940.483336
Topic 1 Word Distribution for lda_2023.0-1.pkl
     Word  Importance
    movie 2400.710913
      man 2045.637633
   spider 1575.099986
     film 1288.462179
character 1174.905887
Topic 2 Word Distribution for lda_2023.0-1.pkl
Word  Importance
 one 2006.689331
life 1862.405055
 get 1693.073090
john 1691.557778
wick 1587.546366
Topic 3 Word Distribution for lda_2023.0-1.pkl
       Word  Importance
       film 4987.864926
     ranbir 4448.268006
   shraddha 4068.801798
performance 2239.810118
     kapoor 1734.452532
Topic 4 Word Distribution for lda_2023.0-1.pkl
  Word  Importance
  film 1762.053820
ranbir 1186.931992
comedy 1028.702892
  bear  775.011141
 movie  696.683061
Topic 5 Word Distribution for lda_2023.0-1.pkl
  Word  Importance
  film 3433.196242
horror 3208.345278
 movie 2888.006858
  good 1857.023836

## Conclusions:

Hm...It seems like there isn't really a lot of use for this particular model in picking up the topics.

It seems like the LDA model is picking up on some themes, but there's also a lot of overlap between the topics. This is a common issue with LDA, as it's an unsupervised method and doesn't have any way of knowing which words are more meaningful or distinct.

I guess I'll have to distinguish between different areas and use the topic keywords as seeds to improve the topic modelling

##Notes for improvement

I think segmenting by 6 months is way too little, I need to consider segmenting topics by groups of 3-5 years, perhaps.

Other methods to consider using apart from LDA:
> Non-negative Matrix Factorization (NMF)

> Correlated Topic Model

> NTM


#RERUN: Do LDA modelling for 7 topics, and group data in 5 years:

Also, I want to probably model each group of 5 years at a time, seeing the output as well

---

##Step 4: Checkpoint for rerunning LDA


In [ ]:
import pandas as pd
import re

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import pickle
from tqdm import tqdm

In [ ]:
#Run this if reinstancing the environment and don't want to go through the above steps:
df_loaded = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/merged_data_pre_LDA.csv')

<ipython-input-5-24c3daef8f3d>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_loaded = pd.read_csv('/content/drive/MyDrive/IMDB Project/review_analysis/data/merged_data_pre_LDA.csv')


In [ ]:
#Make a copy for editing
df = df_loaded.copy()

---

In [ ]:
# There are 2 release dates, time to drop one of them and rename the other one
df.drop('release_date_y', axis=1, inplace=True)
df.rename(columns={'release_date_x': 'release_date'}, inplace=True)

In [ ]:
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date,Genres
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,"Action, Crime, Thriller"
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,"Action, Crime, Thriller"
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,"Action, Crime, Thriller"
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,"Action, Crime, Thriller"
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,"Action, Crime, Thriller"


---

##Step 5: Preparing for LDA - By Date

>Before we perform LDA, we need to prepare our data. Since you want to perform LDA by year and by genre, we need to group our data accordingly.


>We'll start with grouping by every 5 years

In [ ]:
import numpy as np

In [ ]:
# Convert 'release_date' to datetime format
df['release_date'] = pd.to_datetime(df['release_date'])

# Extract year from 'release_date'
df['year'] = df['release_date'].dt.year

In [ ]:
# Create a new column 'year_group' to represent each 5-year period
df['year_group'] = (df['year'] // 5) * 5

# Display the first few rows of the dataframe
df.head()

,Unnamed: 0,reviewer,rating,imdb_id,title,preprocessed_review,release_date,Genres,year,year_group
0,0,bob the moo,NaN,tt0114126,Thunderbolt,"['jackie', 'loh', 'chan', 'motor', 'mechanic',...",1995-08-05,"Action, Crime, Thriller",1995.0,1995.0
1,1,sagacity_,NaN,tt0114126,Thunderbolt,"['one', 'important', 'thing', 'jackie', 'chan'...",1995-08-05,"Action, Crime, Thriller",1995.0,1995.0
2,2,rutt13-1,8.0,tt0114126,Thunderbolt,"['read', 'somewher', 'jackie', 'still', 'recov...",1995-08-05,"Action, Crime, Thriller",1995.0,1995.0
3,3,ma-cortes,6.0,tt0114126,Thunderbolt,"['moving', 'picture', 'deal', 'chan', 'foh', '...",1995-08-05,"Action, Crime, Thriller",1995.0,1995.0
4,4,OllieSuave-007,6.0,tt0114126,Thunderbolt,"['another', 'action-packed', 'movie', 'starrin...",1995-08-05,"Action, Crime, Thriller",1995.0,1995.0


In [ ]:
# Display the counts of reviews for each year
df['year'].value_counts().sort_index()

1980.0     11298
1981.0     12798
1982.0     16254
1983.0     14107
1984.0     16667
1985.0     17463
1986.0     19072
1987.0     20026
1988.0     17486
1989.0     17392
1990.0     27495
1991.0     24855
1992.0     24812
1993.0     22545
1994.0     31987
1995.0     38260
1996.0     36296
1997.0     49665
1998.0     61731
1999.0     90401
2000.0     77857
2001.0     94374
2002.0     95352
2003.0     96669
2004.0     96616
2005.0    107136
2006.0    116344
2007.0    100579
2008.0     95760
2009.0     90108
2010.0     86440
2011.0     87134
2012.0     96922
2013.0    102275
2014.0    114133
2015.0    109020
2016.0    119674
2017.0    154886
2018.0    219290
2019.0    264314
2020.0    188863
2021.0    248634
2022.0    166586
2023.0     48588
2024.0         9
Name: year, dtype: int64

##Step 6: Graphing the count of reviews each year and by group:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# Create a figure and axis for the plots
fig, ax = plt.subplots(2, 1, figsize=(15, 10))

# Change the background color of the plots
for axis in ax:
    axis.set_facecolor('w')

# Plot the distribution of the number of reviews each year
sns.histplot(df['year'], bins=range(1980, 2024), ax=ax[0], kde=False, color='blue')
ax[0].set_title('Distribution of Number of Reviews Each Year')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Number of Reviews')
ax[0].set_xticks(range(1980, 2024))  # Set x-ticks for every single year
ax[0].set_yticks(np.arange(0, df['year'].value_counts().max(), 20000))  # Set y-ticks in increments of 20000
ax[0].set_xticklabels(range(1980, 2024), rotation=90)  # Rotate x-ticks by 90 degrees

# Plot the distribution of the number of reviews every 5 years
sns.histplot(df['year_group'], bins=range(1980, 2024, 5), ax=ax[1], kde=False, color='lightsteelblue')
ax[1].set_title('Distribution of Number of Reviews Every 5 Years')
ax[1].set_xlabel('Year Group')
ax[1].set_ylabel('Number of Reviews')
ax[1].yaxis.set_major_locator(MaxNLocator(nbins='auto', steps=[1, 2, 5, 10]))  # Set y-ticks automatically
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0f}'.format(x)))  # Format y-ticks to remove scientific notation

# Display the plots
plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
from scipy.stats import linregress

# Calculate the number of reviews each year
reviews_per_year = df['year'].value_counts().sort_index()

# Calculate the number of reviews every 5 years
reviews_per_year_group = df['year_group'].value_counts().sort_index()

# Create a figure and axis for the plots
fig, ax = plt.subplots(2, 1, figsize=(15, 10))

# Change the background color of the plots
for axis in ax:
    axis.set_facecolor('w')

# Plot the distribution of the number of reviews each year
sns.histplot(df['year'], bins=range(1980, 2024), ax=ax[0], kde=False, color='blue')
ax[0].set_title('Distribution of Number of Reviews Each Year')
ax[0].set_xlabel('Year')
ax[0].set_ylabel('Number of Reviews')
ax[0].set_xticks(range(1980, 2024))  # Set x-ticks for every single year
ax[0].set_yticks(np.arange(0, df['year'].value_counts().max(), 20000))  # Set y-ticks in increments of 20000
ax[0].set_xticklabels(range(1980, 2024), rotation=90)  # Rotate x-ticks by 90 degrees

# Add a trend line
slope, intercept, r_value, p_value, std_err = linregress(reviews_per_year.index, reviews_per_year.values)
sns.regplot(x=reviews_per_year.index, y=reviews_per_year.values, ax=ax[0], line_kws={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)})
ax[0].legend()

# Plot the distribution of the number of reviews every 5 years
sns.histplot(df['year_group'], bins=range(1980, 2024, 5), ax=ax[1], kde=False, color='lightsteelblue')
ax[1].set_title('Distribution of Number of Reviews Every 5 Years')
ax[1].set_xlabel('Year Group')
ax[1].set_ylabel('Number of Reviews')
ax[1].yaxis.set_major_locator(MaxNLocator(nbins='auto', steps=[1, 2, 5, 10]))  # Set y-ticks automatically
ax[1].yaxis.set_major_formatter(FuncFormatter(lambda x, _: '{:.0f}'.format(x)))  # Format y-ticks to remove scientific notation

# Add a trend line
slope, intercept, r_value, p_value, std_err = linregress(reviews_per_year_group.index, reviews_per_year_group.values)
sns.regplot(x=reviews_per_year_group.index, y=reviews_per_year_group.values, ax=ax[1], line_kws={'label':"y={0:.1f}x+{1:.1f}".format(slope,intercept)})
ax[1].legend()

# Display the plots
plt.tight_layout()
plt.show()


#LDA RESULT

## Step 7 Conducting LDA on each group of 5 years

In [9]:
import plotly.express as px
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
import pickle
from tqdm import tqdm

In [ ]:
from tqdm import tqdm

# Define the 5-year intervals
intervals = [(2014, 2018), (2009, 2013), (2004, 2008), (1999, 2003), (1994, 1998), (1989, 1993), (1984, 1988), (1980, 1983)]

# For each 5-year interval
for start, end in tqdm(intervals):
    # Filter the dataframe to only include rows from the current interval
    df_interval = df[(df['year'] >= start) & (df['year'] <= end)]

    # Initialize a CountVectorizer to convert the text data into a bag-of-words
    count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)

    # Fit and transform the processed reviews
    count_data = count_vectorizer.fit_transform(df_interval['preprocessed_review'])

    # Save the feature names
    feature_names = count_vectorizer.get_feature_names_out()
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/5_year_LDA_models/feature_names_{start}_{end}.pkl', 'wb') as file:
        pickle.dump(feature_names, file)

    # Initialize the LDA model
    lda = LDA(n_components=7, random_state=42)

    # Fit the model to the count data
    lda.fit(count_data)

    # Save the LDA model
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/5_year_LDA_models/lda_{start}_{end}.pkl', 'wb') as file:
        pickle.dump(lda, file)

    # Load the LDA model
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/5_year_LDA_models/lda_{start}_{end}.pkl', 'rb') as f:
        lda = pickle.load(f)

    # Load the feature names
    with open(f'/content/drive/MyDrive/IMDB Project/review_analysis/5_year_LDA_models/feature_names_{start}_{end}.pkl', 'rb') as f:
        feature_names = pickle.load(f)

    # Get the topic-word matrix
    topic_word_matrix = lda.components_

    # Plot the distribution for each topic
    for i in range(topic_word_matrix.shape[0]):
        # Create a dataframe for the current topic
        df_topic = pd.DataFrame({'Word': feature_names, 'Importance': topic_word_matrix[i, :]})

        # Sort the dataframe by importance
        df_topic = df_topic.sort_values('Importance', ascending=False)

        # Create a bar plot using Plotly
        fig = px.bar(df_topic.head(10), x='Word', y='Importance', title=f'Topic {i} Word Distribution for {start}-{end}')
        fig.show()


  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [59:39<6:57:39, 3579.90s/it]

 25%|██▌       | 2/8 [1:42:40<4:59:11, 2991.95s/it]

 38%|███▊      | 3/8 [2:33:57<4:12:34, 3030.90s/it]

 50%|█████     | 4/8 [3:16:17<3:09:08, 2837.14s/it]

 62%|██████▎   | 5/8 [3:35:46<1:51:47, 2235.70s/it]

 75%|███████▌  | 6/8 [3:46:23<56:23, 1691.97s/it]  

 88%|████████▊ | 7/8 [3:54:45<21:42, 1302.93s/it]

100%|██████████| 8/8 [3:59:38<00:00, 1797.32s/it]


## Step 8:

Conducting LDA on ALL the years together

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import pickle

# Initialize a CountVectorizer to convert the text data into a bag-of-words
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)

# Fit and transform the processed reviews
count_data = count_vectorizer.fit_transform(df['preprocessed_review'])

# Save the feature names
feature_names = count_vectorizer.get_feature_names_out()
with open('/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/feature_names_all_years.pkl', 'wb') as file:
    pickle.dump(feature_names, file)

# Initialize the LDA model
lda = LDA(n_components=7, random_state=42)

# Fit the model to the count data
lda.fit(count_data)

# Save the LDA model
with open('/content/drive/MyDrive/IMDB Project/review_analysis/LDA_models/lda_all_years.pkl', 'wb') as file:
    pickle.dump(lda, file)
